In [1]:
##read Data Base
from pymongo import MongoClient
from bson.objectid import ObjectId
uri = "mongodb://localhost:27017/database" #mongodb://<user_name>:<user_password>@ds<xxxxxx>.mlab.com:<xxxxx>/<database_name>
conn = MongoClient(uri)
db = conn.PaperData
#Spectrogram_Collection  =  db.get_collection('AudioSpectrogram_100thTag')
LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag')

In [2]:
import json 
import codecs
import numpy as np
def read_json(filename):
    with codecs.open(filename,'r',encoding = 'utf8') as infile:
        return np.array(json.load(infile))

_num_Tags =50   
x_train = read_json('x_train_'+str(_num_Tags)+'d.json').tolist()
x_test = read_json('x_test_'+str(_num_Tags)+'d.json').tolist()


In [10]:
x_List = x_test + x_train

In [11]:
Text_x = np.array(LineCNN_Collection.find_one({"Filename":x_List[0]})['LineMatrix'])

In [12]:
Text_x.max()

1.2679469585418701

In [13]:
Text_x.min()

-1.4294503927230835

In [14]:
WV_max = 7.682228
WV_min = -9.127636
WV_len = WV_max - WV_min

In [15]:
len(x_List)

42838

In [20]:
for filename in x_List:
    try:
        Text_x = np.array(LineCNN_Collection.find_one({"Filename":filename})['LineMatrix'])

    #print(np_max,np_min)
        Text_x = (Text_x - WV_min)/WV_len
        LineCNN_Collection.update_one({"Filename":filename},{"$set":{'LineMatrix':Text_x.tolist()},"$currentDate":{"lastModified":True} })
    except:
        print(filename)
    progressBar(x_List.index(filename),1,len(x_List)+1)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>[100%]42839/42839

In [21]:
Text_x = np.array(LineCNN_Collection.find_one({"Filename":"Jesse Sykes & the Sweet Hereafter_The Air Is Thin"})['LineMatrix'])


In [22]:
Text_x.shape

(33, 25, 100)

In [24]:
Text_x.max()

0.6258050174302724

In [25]:
Text_x.min()

0.466438424135411

In [23]:
import sys

In [17]:
def progressBar(index_progress, start, end):
    i = int((index_progress)/(end-start)*100)
    k = i + 1
    the_str = '>'*(i//2)+' '*((100-k)//2)
    sys.stdout.write('\r'+the_str+'[%s%%]'%(i+1)+ str(index_progress+1+start) + '/' + str(end))
    sys.stdout.flush()

In [49]:
len(the_list)

1

In [27]:
1300/60

21.666666666666668

In [6]:
import sys
def progressBar(index_progress, start, end):
    i = int((index_progress)/(end-start)*100)
    k = i + 1
    the_str = '>'*(i//2)+' '*((100-k)//2)
    sys.stdout.write('\r'+the_str+'[%s%%]'%(i+1)+ str(index_progress+1+start) + '/' + str(end))
    sys.stdout.flush()

In [11]:
## Padding for batch

import codecs
import numpy as np
import json 
def read_json(filename):
    with codecs.open(filename,'r',encoding = 'utf8') as infile:
        return np.array(json.load(infile))
_num_Tags = 50
x_train = read_json('x_train_'+str(_num_Tags)+'d_sub.json')
x_test = read_json('x_test_'+str(_num_Tags)+'d_sub.json')
X_List = x_train.tolist()+x_test.tolist()

from pymongo import MongoClient
from bson.objectid import ObjectId
from multiprocessing import Process
from multiprocessing import Pool, TimeoutError
uri = "mongodb://localhost:27017/database" #mongodb://<user_name>:<user_password>@ds<xxxxxx>.mlab.com:<xxxxx>/<database_name>
client = MongoClient(uri)


Max_len = 80

Too_Long_Song = []
# Each child process attempts to copy a global MongoClient
# created in the parent process. Never do this.
def func(File_List,max_len,start,end):
    db = client.PaperData
    
    for filename in File_List:
        progressBar(File_List.index(filename),start,end)
        LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag')
        Text = LineCNN_Collection.find_one({"Filename":filename})['LineMatrix']
        
        if len(Text) > Max_len:
            Too_Long_Song.append(filename)
            continue
        else:
            while len(Text) < Max_len:
                Text.append(np.zeros(shape = (25,100)).tolist())
            P_LineCNN_Collection = db.get_collection('LineCNNMatrix_100thTag_80p')
            P_LineCNN_Collection.insert_one({"Filename":filename,'LineMatrix':Text})

'        \n        else:\n            while len(Text) < Max_len:\n                Text.append(np.zeros(shape = (25,100)).tolist())\n            P_LineCNN_Collection = db.get_collection(\'LineCNNMatrix_100thTag_80p\')\n            P_LineCNN_Collection.insert_one({"Filename":filename,\'LineMatrix\':Text})\n'

In [26]:
Too_Long_Song = []

db = client.PaperData
    
for filename in X_List:
    progressBar(X_List.index(filename),0,len(X_List))
    LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag')
    Text = LineCNN_Collection.find_one({"Filename":filename})['LineMatrix']
        
    if len(Text) > Max_len:
        Too_Long_Song.append(filename)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>[100%]26129/26129

In [13]:
func(X_List[0:1],Max_len,0,1,Too_Long_Song)

                                                 [1%]1/1

In [14]:
Process_List = []
Process_List.append(Process(target=func, args=(X_List[0:2000],Max_len,0,2000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[2000:4000],Max_len,2000,4000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[4000:6000],Max_len,4000,6000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[6000:8000],Max_len,6000,8000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[8000:10000],Max_len,8000,10000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[10000:12000],Max_len,10000,12000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[12000:14000],Max_len,12000,14000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[14000:16000],Max_len,14000,16000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[16000:18000],Max_len,16000,18000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[18000:20000],Max_len,18000,20000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[20000:22000],Max_len,20000,22000,Too_Long_Song)))
Process_List.append(Process(target=func, args=(X_List[22000:len(X_List)],Max_len,22000,len(X_List),Too_Long_Song)))

In [15]:
for p in Process_List:
    p.start()

                                                 [1%]1/2000

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


                                                 [1%]4001/6000

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


                                                 [1%]6001/8000

...

                                                 [1%]8002/100000

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


                                                 [1%]8003/100000

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>[100%]26129/26129

In [16]:
for p in Process_List:
    p.join()
#db.close()

In [6]:
db =client.PaperData

In [19]:
np.zeros(shape = (25,100)).shape

(25, 100)

In [18]:
db = client.PaperData
LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag')
P_LineCNN_Collection = db.get_collection('LineCNNMatrix_100thTag_80p')
#Text = LineCNN_Collection.find_one({"Filename":X_List[0]})['LineMatrix']

In [12]:
db.collection_names()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['AudioSpectrogram',
 'system.indexes',
 'AudioSpectrogram_100thTag',
 'LineCNNMatrix_100thTag',
 'LineCNNMatrix_100thTag_80p']

In [25]:
while len(Text) < Max_len:
    Text.append(np.zeros(shape = (25,100)).tolist())

In [26]:
np.array(Text).shape

(332, 25, 100)

<bound method Collection.count of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PaperData'), 'LineCNNMatrix_100thTag_80p')>

In [27]:
len(Too_Long_Song)

453

In [28]:
x_train = read_json('x_train_'+str(_num_Tags)+'d_sub.json')
x_test = read_json('x_test_'+str(_num_Tags)+'d_sub.json')
y_train = read_json('y_train_'+str(_num_Tags)+'d_sub.json')
y_test = read_json('y_test_'+str(_num_Tags)+'d_sub.json')

In [29]:
y_List = np.array(y_train.tolist() + y_test.tolist())
x_List = np.array(x_train.tolist() + x_test.tolist())

In [30]:
X_subset=[]
Y_subset=[]

for i in range(len(x_List)):
    if x_List[i] not in Too_Long_Song:
        X_subset.append(x_List[i].tolist())
        Y_subset.append(y_List[i].tolist())

In [31]:
from sklearn.model_selection import train_test_split
x_sub_train, x_sub_test , y_sub_train, y_sub_test = train_test_split(X_subset,Y_subset,test_size=0.2, random_state=100)

In [32]:
len(x_sub_test)

5136

In [33]:
len(x_sub_train)

20540

In [34]:
len(y_sub_test)

5136

In [35]:
len(y_sub_train)

20540

In [36]:
type(y_sub_train[0])

list

In [37]:
def write_json(filename,data):
    with codecs.open(filename,'w',encoding = 'utf8') as infile:
        json.dump(data,infile)
    return len(data)

In [38]:
write_json('x_train_'+str(_num_Tags)+'d_80p_sub.json',x_sub_train)

20540

In [39]:
write_json('y_train_'+str(_num_Tags)+'d_80p_sub.json',y_sub_train)

20540

In [40]:
write_json('x_test_'+str(_num_Tags)+'d_80p_sub.json',x_sub_test)

5136

In [41]:
write_json('y_test_'+str(_num_Tags)+'d_80p_sub.json',y_sub_test)

5136